In [ ]:
import os
# Change root directory
os.chdir("../src/")


In [2]:
from agent_design_pattern.agent import AgentMessage, LLMChain
from agent_design_pattern.orchestration import ReflectionAgent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

/data/agent_design_pattern/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CasualOllamaChain(LLMChain):
    def __init__(self, model, system_prompt, user_prompt_template = "{query}", **kargs):
        super().__init__()
        self.llm = ChatOllama(model=model, reasoning=False, **kargs) if isinstance(model, str) else model
        self.prompt = ChatPromptTemplate([
            ("system", system_prompt),
            ("human", user_prompt_template)
        ])
        self.chain = self.prompt | self.llm | StrOutputParser()

    def invoke(self, message: AgentMessage, **kwargs) -> AgentMessage:
        response = self.chain.invoke(message.to_dict(), **kwargs)
        print(response)
        # Snippet to remove thinking
        if "</think>" in response and "<think>" not in response:
            response = "<think>" + response
        response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
        message.response = response

        message.execution_result = "success"
        return message

In [4]:
system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
"""
user_prompt_task = "{query}"
task_chain = CasualOllamaChain("qwen3:4b-thinking-2507-q4_K_M", system_prompt_task, user_prompt_task, base_url="192.168.55.1::11434")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
user_prompt_reflection = "Input query: {query}\n\nFunction implementation: {artifact_response}"
reflection_chain = CasualOllamaChain(task_chain.llm, system_prompt_reflection, user_prompt_reflection, base_url="192.168.55.1::11434")  # use the same model for self-reflection

def state_callback(state: str):
    print(state)
reflection_agent = ReflectionAgent(task_chain, reflection_chain, state_change_callback=state_callback)

In [5]:
reflection_agent.execute(AgentMessage(query="Hello"), AgentMessage(query="Hello"))

running
Okay, let's see. The user wants me to write a Python function with specific requirements. They mentioned that the function should be clear, have typed arguments and return values, not too long (under 80 lines), split into smaller functions if needed, and have docstrings with examples.

First, I need to figure out what function they want. Wait, the user said "Hello" in their message. Hmm, maybe they just said "Hello" to start the conversation, but the actual task is to write a function. Wait, looking back, the user's message is "Hello", which might be a test. Wait, the problem statement says: "You task is to write a python function and return the implementation of the function."

Wait, maybe the user intended to ask for a specific function but just said "Hello". Oh, but the problem says "Hello" is the user's message. Wait, no—the original instruction says: "Hello" is the user's input here. Wait, the problem says: "You task is to write a python function..." but the user's message

AgentMessage(query='Hello', origin='ReflectionAgent_1', response='\n\n```python\ndef hello(name: str) -> str:\n    """\n    Returns a greeting message for the given name.\n\n    Args:\n        name: str\n            The name to greet.\n\n    Returns:\n        str\n            A formatted greeting string.\n\n    Example:\n        >>> hello("Alice")\n        \'Hello, Alice!\'\n    """\n    return f"Hello, {name}!"\n```', artifact={'response': '\n\nHere\'s a simple and clear Python function that meets all the specified requirements:\n\n```python\ndef hello(name: str) -> str:\n    """\n    Return a greeting message with the given name.\n\n    This function takes a name as input and returns a formatted greeting string.\n\n    Example:\n        >>> hello("Alice")\n        \'Hello, Alice!\'\n    """\n    return f"Hello, {name}!"\n```\n\n**Explanation**:  \nThis function is designed to be:\n1. Clear and easy to understand (simple string formatting)\n2. Properly typed with `name` as `str` and r